In [ ]:
!pip install wandb

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(project="Cifar100", entity="gharabadiyan")

wandb: Currently logged in as: gharabadiyan. Use `wandb login --relogin` to force relogin


In [5]:
cifar100 = tf.keras.datasets.cifar100

(X_train, Y_train), (X_test, Y_test) = cifar100.load_data()

X_test = X_test / 255.0

X_train,X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=24)

X_train = X_train / 255.0

169017344/169001437 [==============================] - 2s 0us/step


## MLP

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32,3)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(100,activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=25,validation_data=(X_val,Y_val),callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/25


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1250/1250 [==============================] - 7s 3ms/step - loss: 4.2370 - accuracy: 0.0488 - val_loss: 293.4640 - val_accuracy: 0.0711 - _timestamp: 1654668878.0000 - _runtime: 70.0000
Epoch 2/25
1250/1250 [==============================] - 4s 3ms/step - loss: 3.8731 - accuracy: 0.0966 - val_loss: 387.4752 - val_accuracy: 0.1024 - _timestamp: 1654668882.0000 - _runtime: 74.0000
Epoch 3/25
1250/1250 [==============================] - 4s 3ms/step - loss: 3.6676 - accuracy: 0.1361 - val_loss: 448.3227 - val_accuracy: 0.1150 - _timestamp: 1654668886.0000 - _runtime: 78.0000
Epoch 4/25
1250/1250 [==============================] - 4s 3ms/step - loss: 3.5456 - accuracy: 0.1577 - val_loss: 455.9567 - val_accuracy: 0.1462 - _timestamp: 1654668890.0000 - _runtime: 82.0000
Epoch 5/25
1250/1250 [==============================] - 4s 3ms/step - loss: 3.4536 - accuracy: 0.1752 - val_loss: 479.6448 - val_accuracy: 0.1368 - _timestamp: 1654668894.0000 - _runtime: 86.0000
Epoch 6/25
1250/1250 [=========

In [7]:
model.evaluate(X_test, Y_test)

 39/313 [==>...........................] - ETA: 0s - loss: 3.6530 - accuracy: 0.1915

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


313/313 [==============================] - 1s 3ms/step - loss: 3.6190 - accuracy: 0.1956


[3.6190261840820312, 0.1956000030040741]

In [8]:
model.save('MLP_Cifar100.h5')

## MLP + CNN

In [9]:
model = tf.keras.models.Sequential([
                                    
        #استخراج ویژگی                           
        layers.Conv2D(32,(3,3),activation='relu',input_shape=(32, 32,3)),
        layers.Conv2D(32,(3,3),activation='relu'),                            
        layers.MaxPool2D((2, 2)),
        layers.Conv2D(64,(3,3),activation='relu'),
        layers.Conv2D(64,(3,3),activation='relu'),
        layers.MaxPool2D((2, 2)),

        layers.Flatten(),

        #همون شبکه قدیمی
        tf.keras.layers.Dense(48, activation='relu'),
        tf.keras.layers.Dense(100)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.fit(X_train,Y_train,epochs=15,validation_data=(X_val,Y_val),callbacks=[WandbCallback()])

Epoch 1/15
1250/1250 [==============================] - 16s 5ms/step - loss: 4.1251 - accuracy: 0.0662 - val_loss: 500.7497 - val_accuracy: 0.0834 - _timestamp: 1654669025.0000 - _runtime: 217.0000
Epoch 2/15
1250/1250 [==============================] - 6s 5ms/step - loss: 3.4743 - accuracy: 0.1702 - val_loss: 493.6456 - val_accuracy: 0.1157 - _timestamp: 1654669031.0000 - _runtime: 223.0000
Epoch 3/15
1250/1250 [==============================] - 6s 5ms/step - loss: 3.1335 - accuracy: 0.2341 - val_loss: 525.6008 - val_accuracy: 0.1208 - _timestamp: 1654669037.0000 - _runtime: 229.0000
Epoch 4/15
1250/1250 [==============================] - 5s 4ms/step - loss: 2.9054 - accuracy: 0.2769 - val_loss: 450.6278 - val_accuracy: 0.1302 - _timestamp: 1654669042.0000 - _runtime: 234.0000
Epoch 5/15
1250/1250 [==============================] - 5s 4ms/step - loss: 2.7164 - accuracy: 0.3126 - val_loss: 388.2565 - val_accuracy: 0.1657 - _timestamp: 1654669047.0000 - _runtime: 239.0000
Epoch 6/15
125

In [12]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 2.7669 - accuracy: 0.3365


[2.7669010162353516, 0.33649998903274536]

In [13]:
model.save('MLP_CNN_Cifar100.h5')